In [9]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

character(0)

In [10]:
Time series Forcasting Kaggle Competation.

Step 1- Install Packages

library(tidyverse) # metapackage of all tidyverse packages

library(forecast)
library(dplyr)
library(lubridate)
library(ggplot2)
library(tseries)
library(zoo)
library(prophet)



# Import Train and Test Data

train_data <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/train.csv')
test_data <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/test.csv')
holidays <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/holidays.csv')


# Summary Statistics
head(train_data);dim(train_data)
head(test_data);dim(test_data)
head(holidays);dim(holidays)
summary(train_data);summary(test_data);summary(holidays)

# Data Cleaning & Prep - Convert String to Date format code

str(train_data$date)
str(test_data$date)
str(holidays$Date)

#Convert String to Date format code
train_data$date = as.Date(train_data$date)
test_data$date = as.Date(test_data$date)
holidays$Date = as.Date(holidays$Date)
names(holidays$Date)<-c('date','holidays')
class(train_data$date);class(test_data$date);class(holidays$Date)
str(train_data$date)
str(test_data$date)
str(holidays$Date)


# Data Cleaning & Prep - Identify and Treat Missing Dates with mean of previous years
# Sort the data by date
train_data <- train_data %>% arrange(date)
test_data <- test_data %>% arrange(date)

nrow(train_data)
sum(is.na(train_data)) 

nrow(test_data)
sum(is.na(test_data)) 

# Check for missing values in the date column of train_data and count them
missing_train_data <- sum(is.na(train_data$date))

# Check for missing values in the date column of test_data and count them
missing_test_data <- sum(is.na(test_data$date))

# Print the number of missing values in each data frame
cat("Missing values in train_data:", missing_train_data, "\n")
cat("Missing values in test_data:", missing_test_data, "\n")


#After Date conversion
head(holidays)
head(train_data)
head(test_data)

# Data Cleaning & Prep - Identify and Treat Outliers (tsclean)
train_ts = ts(train_data$transactions, frequency = 365, start = c(2013, 1))
cleaned_data = tsclean(train_ts)
plot(train_ts)


# EDA - STL plot (periodic)
stl_result <- stl(cleaned_data, s.window = "periodic")
plot(stl_result)

# EDA - decompose plot (additive)
decomp_result <- decompose(cleaned_data, type = "additive")
plot(decomp_result)


# ACF and PACF Plots
acf(cleaned_data)
pacf(cleaned_data)


# Perform ADF Test -Augmented Dickey-Fuller Test
adf_result <- adf.test(cleaned_data)
summary(adf_result)
print(adf_result)

# ARIMA Model
model_arima <- arima(cleaned_data, order = c(1,0,1))
summary(model_arima)

# MA term ? - ACF plot 
par(mfrow=c(1, 3))
acf(cleaned_data,lag.max =20) # Mostly all the correlated time period will be in last 20 time frame.
acf(cleaned_data,lag.max =200)
acf(cleaned_data,lag.max =400)
par(mfrow=c(1, 1))


# AR term - PACF plot
par(mfrow=c(1, 3))
pacf(cleaned_data, lag.max = 20)
pacf(cleaned_data, lag.max = 200)
pacf(cleaned_data, lag.max = 400)
par(mfrow=c(1, 1))


# Check for I term - to check if time series is stationary or not 
adf.test(cleaned_data)
# p-value is greater than 0.05 => TS in not stationary
I =0
#ARIMA = ( ?, 0, ?)



# ARIMAX Model - with auto.arima (Holiday)
model <- auto.arima(cleaned_data)
summary(model)

# Other Model

# ses model
# Fit a Simple Exponential Smoothing (SES) model to the training data
mod_ses <- ses(cleaned_data)

# Make forecasts for the test data
pred_ses<-forecast(mod_ses, length(test_data))

# Calculate accuracy metrics for the fitted values
accuracy(mod_ses$fitted, cleaned_data)  # Replace 'cleaned_data' with your training data

# Print the forecasted values
print(pred_ses)
# Plot the forecasted values
plot(pred_ses)


# holt model 
# Fit a Holt's Exponential Smoothing (Holt) model
mod_holt <- holt(cleaned_data)

# Make forecasts for the test data
pred_holt <- forecast(mod_holt, h = length(test_data))

# Calculate accuracy metrics for the fitted values
accuracy(mod_holt$fitted, cleaned_data)  # Replace 'cleaned_data' with your training data

# Print the forecasted values
print(pred_holt)

# Plot the forecasted values
plot(pred_holt)


# holt winter model 
mod_hw<-HoltWinters(cleaned_data)
pred_hw<-forecast(mod_hw,h=length(test_data))

# Calculate accuracy metrics for the fitted values
accuracy(mod_hw$fitted, cleaned_data)  # Replace 'cleaned_data' with  training data

# Print the forecasted values
print(pred_hw)
# Plot the forecasted values
plot(pred_hw)


# Calculate Performance Metrics
# 1 point for (ME, RMSE, MAE, MPE, MAPE)
model <- auto.arima(cleaned_data)
summary(model)


# ARIMAX Model - with auto.arima (Holiday)
# Rename the 'Date' column in the 'holidays' dataset to 'date'
names(holidays)[names(holidays) == "Date"] <- "date"

# Merge the cleaned_data with holidays
merged_data <- merge(cleaned_data, holidays, by.x = "date", by.y = "date", all.x = TRUE)

# Replace NAs in the 'Holidays' column with 0 (assuming no holiday on those days)
merged_data$Holidays[is.na(merged_data$Holidays)] <- 0

# Fit an ARIMAX model
library(forecast)
model <- auto.arima(merged_data$transactions, xreg = merged_data$Holidays)
summary(model)



# Forecast using Model
# Replace NAs in the 'Holidays' column with 0 (assuming no holiday on those days)
merged_data$Holidays[is.na(merged_data$Holidays)] <- 0

# Fit an ARIMAX model
model <- auto.arima(merged_data$transactions, xreg = merged_data$Holidays)

# Make forecasts
forecast_horizon <- 12  # Adjust this value for your desired forecast horizon
forecasts <- forecast(model, xreg = merged_data$Holidays, h = forecast_horizon)

# Print the forecasts
print(forecasts)

# Plot the forecasts
plot(forecasts)


# plot with Actual and Predicted Values
# Sample data (replace with your own dataset)
set.seed(42)
date_sequence <- seq.Date(from = as.Date("2023-01-01"), by = "days", length.out = 100)
data <- data.frame(Date = date_sequence, Value = rnorm(100))

# Split the data into training and testing sets (70% train, 30% test)
set.seed(42)  # For reproducibility
split_index <- floor(0.7 * nrow(data))
train_data <- data[1:split_index, ]
test_data <- data[(split_index + 1):nrow(data), ]

# Assuming you're using linear regression for forecasting
model <- lm(Value ~ as.numeric(1:nrow(train_data)), data = train_data)

# Forecast the testing data
forecasted_values <- predict(model, newdata = data.frame(as.numeric(1:nrow(test_data))))

# Create a subset of forecasted values to match the number of rows in test_data
forecasted_values_subset <- forecasted_values[1:nrow(test_data)]

# Add forecasted values to the test_data data frame
test_data$Forecasted <- forecasted_values_subset

# Plot the actual vs. forecasted data
library(ggplot2)

ggplot(test_data, aes(x = Date)) +
  geom_line(aes(y = Value, color = "Actual"), size = 1) +
  geom_line(aes(y = Forecasted, color = "Forecasted"), size = 1, linetype = "dashed") +
  labs(x = "Date", y = "Value", title = "Actual vs. Forecasted Data") +
  scale_color_manual(values = c("Actual" = "green", "Forecasted" = "orange")) +
  theme_minimal()


ERROR: Error in parse(text = x, srcfile = src): <text>:1:6: unexpected symbol
1: Time series
         ^


In [11]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load
​
library(tidyverse) # metapackage of all tidyverse packages
​
library(forecast)
library(dplyr)
library(lubridate)
library(ggplot2)
library(tseries)
library(zoo)
library(prophet)
​
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
​
list.files(path = "../input")
​
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: Rcpp

Loading required package: rlang


Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice


'ipba-17-time-series-forecasting-png'
add Codeadd Markdown
train_ds <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/train.csv')
test_ds <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/test.csv')
holidays <- read.csv('/kaggle/input/ipba-17-time-series-forecasting-png/holidays.csv')
add Codeadd Markdown
# Summary Statistics
head(train_ds);dim(train_ds)
head(test_ds);dim(test_ds)
head(holidays);dim(holidays)
summary(train_ds);summary(test_ds);summary(holidays)
A data.frame: 6 × 2
date	transactions
<chr>	<int>
1	01-01-2013	770
2	02-01-2013	93215
3	03-01-2013	78504
4	04-01-2013	78494
5	05-01-2013	93573
6	06-01-2013	90464
14552
A data.frame: 6 × 2
date	id
<chr>	<int>
1	01-01-2017	0
2	02-01-2017	1
3	03-01-2017	2
4	04-01-2017	3
5	05-01-2017	4
6	06-01-2017	5
2272
A data.frame: 6 × 2
Date	Holidays
<chr>	<int>
1	02-03-2012	1
2	01-04-2012	1
3	12-04-2012	1
4	14-04-2012	1
5	21-04-2012	1
6	12-05-2012	1
3122
     date            transactions   
 Length:1455        Min.   :   770  
 Class :character   1st Qu.: 75596  
 Mode  :character   Median : 80387  
                    Mean   : 83491  
                    3rd Qu.: 90166  
                    Max.   :171169  
     date                 id       
 Length:227         Min.   :  0.0  
 Class :character   1st Qu.: 56.5  
 Mode  :character   Median :113.0  
                    Mean   :113.0  
                    3rd Qu.:169.5  
                    Max.   :226.0  
     Date              Holidays    
 Length:312         Min.   :1.000  
 Class :character   1st Qu.:1.000  
 Mode  :character   Median :1.000  
                    Mean   :1.122  
                    3rd Qu.:1.000  
                    Max.   :4.000  
add Codeadd Markdown
#Data Cleaning & Preparation - Convert String to Date format code
str(train_ds$date)
str(test_ds$date)
str(holidays$Date)
​
#Convert String to Date format code
train_ds$date = as.Date(train_ds$date)
test_ds$date = as.Date(test_ds$date)
holidays$Date = as.Date(holidays$Date)
names(holidays$Date)<-c('date','holidays')
class(train_ds$date);class(test_ds$date);class(holidays$Date)
str(train_ds$date)
str(test_ds$date)
str(holidays$Date)
 chr [1:1455] "01-01-2013" "02-01-2013" "03-01-2013" "04-01-2013" ...
 chr [1:227] "01-01-2017" "02-01-2017" "03-01-2017" "04-01-2017" ...
 chr [1:312] "02-03-2012" "01-04-2012" "12-04-2012" "14-04-2012" ...
'Date'
'Date'
'Date'
 Date[1:1455], format: "1-01-20" "2-01-20" "3-01-20" "4-01-20" "5-01-20" "6-01-20" "7-01-20" ...
 Date[1:227], format: "1-01-20" "2-01-20" "3-01-20" "4-01-20" "5-01-20" "6-01-20" "7-01-20" ...
 Date[1:312], format: "2-03-20" "1-04-20" "12-04-20" "14-04-20" "21-04-20" "12-05-20" ...
add Codeadd Markdown
#Identifying and Treating Missing Dates with mean of previous years
# Sort the data by date
train_ds <- train_ds %>% arrange(date)
test_ds <- test_ds %>% arrange(date)
​
nrow(train_ds)
sum(is.na(train_ds)) 
​
nrow(test_ds)
sum(is.na(test_ds)) 
​
# Check for missing values in the date column of train_data and count them
missing_train_data <- sum(is.na(train_ds$date))
​
# Check for missing values in the date column of test_data and count them
missing_test_data <- sum(is.na(test_ds$date))
​
# Print the number of missing values in each data frame
cat("Missing values in train_ds:", missing_train_data, "\n")
cat("Missing values in test_ds:", missing_test_data, "\n")
1455
0
227
0
Missing values in train_ds: 0 
Missing values in test_ds: 0 
add Codeadd Markdown
#After Date conversion
head(holidays)
head(train_ds)
head(test_ds)
A data.frame: 6 × 2
Date	Holidays
<date>	<int>
1	2-03-20	1
2	1-04-20	1
3	12-04-20	1
4	14-04-20	1
5	21-04-20	1
6	12-05-20	1
A data.frame: 6 × 2
date	transactions
<date>	<int>
1	1-01-20	770
2	1-01-20	1327
3	1-01-20	2202
4	1-02-20	78302
5	1-02-20	100397
6	1-02-20	98364
A data.frame: 6 × 2
date	id
<date>	<int>
1	1-01-20	0
2	1-02-20	31
3	1-03-20	59
4	1-04-20	90
5	1-05-20	120
6	1-06-20	151
add Codeadd Markdown
#Identifying and Treating Outliers (tsclean)
train_ts = ts(train_ds$transactions, frequency = 365, start = c(2013, 1))
cleaned_data = tsclean(train_ts)
plot(train_ts)

add Codeadd Markdown
#EDA - STL plot (periodic)
stl_result <- stl(cleaned_data, s.window = "periodic")
plot(stl_result)

add Codeadd Markdown
#EDA - decompose plot (additive)
decomp_result <- decompose(cleaned_data, type = "additive")
plot(decomp_result)

add Codeadd Markdown
#ACF Plots
acf(cleaned_data)
​
#PACF Plots
pacf(cleaned_data)


add Codeadd Markdown
#Performing ADF Test -Augmented Dickey-Fuller Test
adf_result <- adf.test(cleaned_data)
summary(adf_result)
print(adf_result)
Warning message in adf.test(cleaned_data):
“p-value smaller than printed p-value”
            Length Class  Mode     
statistic   1      -none- numeric  
parameter   1      -none- numeric  
alternative 1      -none- character
p.value     1      -none- numeric  
method      1      -none- character
data.name   1      -none- character

	Augmented Dickey-Fuller Test

data:  cleaned_data
Dickey-Fuller = -6.9201, Lag order = 11, p-value = 0.01
alternative hypothesis: stationary

add Codeadd Markdown
#ARIMA Model
model_arima <- arima(cleaned_data, order = c(1,0,1))
summary(model_arima)

Call:
arima(x = cleaned_data, order = c(1, 0, 1))

Coefficients:
         ar1      ma1   intercept
      0.4503  -0.1197  83152.1873
s.e.  0.0599   0.0648    394.8403

sigma^2 estimated as 88541924:  log likelihood = -15377.13,  aic = 30762.27

Training set error measures:
                   ME     RMSE      MAE       MPE     MAPE      MASE
Training set 2.341404 9409.672 7476.893 -1.179682 8.848857 0.8821084
                     ACF1
Training set -0.002007772
add Codeadd Markdown
# MA term ? - ACF plot 
​
par(mfrow=c(1, 3))
acf(cleaned_data,lag.max =20) # Mostly all the correlated time period will be in last 20 time frame.
acf(cleaned_data,lag.max =200)
acf(cleaned_data,lag.max =400)
par(mfrow=c(1, 1))

add Codeadd Markdown
# AR term - PACF plot
par(mfrow=c(1, 3))
pacf(cleaned_data, lag.max = 20)
pacf(cleaned_data, lag.max = 200)
pacf(cleaned_data, lag.max = 400)
par(mfrow=c(1, 1))

add Codeadd Markdown
# Check for I term - to check if time series is stationary or not 
adf.test(cleaned_data)
# p-value is greater than 0.05 => TS in not stationary
I =0
Warning message in adf.test(cleaned_data):
“p-value smaller than printed p-value”

	Augmented Dickey-Fuller Test

data:  cleaned_data
Dickey-Fuller = -6.9201, Lag order = 11, p-value = 0.01
alternative hypothesis: stationary
add Codeadd Markdown
#ARIMAX Model - with auto.arima (Holiday)
model <- auto.arima(cleaned_data)
summary(model)
Series: cleaned_data 
ARIMA(4,1,5) 

Coefficients:
          ar1     ar2      ar3      ar4      ma1      ma2     ma3     ma4
      -0.2481  1.1183  -0.0584  -0.8344  -0.4964  -1.4414  0.8160  0.7541
s.e.   0.0397  0.0324   0.0457   0.0358   0.0565   0.0277  0.1027  0.0313
          ma5
      -0.6035
s.e.   0.0566

sigma^2 = 79440365:  log likelihood = -15285.32
AIC=30590.64   AICc=30590.79   BIC=30643.46

Training set error measures:
                   ME     RMSE      MAE        MPE     MAPE     MASE       ACF1
Training set 37.36595 8882.251 6808.448 -0.9846441 8.041626 0.597526 0.04088999
add Codeadd Markdown
#Other Model - holt winter model ,ses model,holt model
# holt winter model 
mod_hw<-HoltWinters(cleaned_data)
pred_hw<-forecast(mod_hw,h=length(test_ds))
#accuracy(mod_hw$fitted, ts_train)
​
# Print the forecasted values
print(pred_hw)
# Plot the forecasted values
plot(pred_hw)
          Point Forecast    Lo 80    Hi 80    Lo 95    Hi 95
2016.9863       85581.41 70329.63 100833.2 62255.82 108907.0
2016.9890       87895.10 72641.86 103148.3 64567.28 111222.9

add Codeadd Markdown
# ses model
# Fit a Simple Exponential Smoothing (SES) model to the training data
mod_ses <- ses(cleaned_data)
​
# Make forecasts for the test data
pred_ses<-forecast(mod_ses, length(test_ds))
​
# Calculate accuracy metrics for the fitted values
accuracy(mod_ses$fitted, cleaned_data)  # Replace 'cleaned_data' with your training data
​
# Print the forecasted values
print(pred_ses)
# Plot the forecasted values
plot(pred_ses)
A matrix: 1 × 7 of type dbl
ME	RMSE	MAE	MPE	MAPE	ACF1	Theil's U
Test set	36.69142	9825.839	7855.087	-1.215635	9.246583	0.2964826	0.856464
          Point Forecast    Lo 80    Hi 80    Lo 95    Hi 95
2016.9863       89721.95 77120.97 102322.9 70450.41 108993.5
2016.9890       89721.95 77116.88 102327.0 70444.16 108999.7
2016.9918       89721.95 77112.80 102331.1 70437.91 109006.0
2016.9945       89721.95 77108.71 102335.2 70431.66 109012.2
2016.9973       89721.95 77104.63 102339.3 70425.41 109018.5
2017.0000       89721.95 77100.54 102343.4 70419.17 109024.7
2017.0027       89721.95 77096.46 102347.4 70412.93 109031.0
2017.0055       89721.95 77092.38 102351.5 70406.69 109037.2
2017.0082       89721.95 77088.30 102355.6 70400.45 109043.5
2017.0110       89721.95 77084.22 102359.7 70394.21 109049.7

add Codeadd Markdown
# holt model 
# Fit a Holt's Exponential Smoothing (Holt) model
mod_holt <- holt(cleaned_data)
​
# Make forecasts for the test data
pred_holt <- forecast(mod_holt, h = length(test_ds))
​
# Calculate accuracy metrics for the fitted values
accuracy(mod_holt$fitted, cleaned_data)  # Replace 'cleaned_data' with your training data
​
# Print the forecasted values
print(pred_holt)
​
# Plot the forecasted values
plot(pred_holt)
A matrix: 1 × 7 of type dbl
ME	RMSE	MAE	MPE	MAPE	ACF1	Theil's U
Test set	-237.7993	10230.78	8199.628	-1.509617	9.694957	0.2613101	0.9072103
          Point Forecast    Lo 80    Hi 80    Lo 95    Hi 95
2016.9863       107309.2 94179.83 120438.5 87229.58 127388.7
2016.9890       107887.0 94582.46 121191.6 87539.46 128234.6


ERROR: Error in parse(text = x, srcfile = src): <text>:4:1: unexpected input
3: # For example, here's a helpful package to load
4: ​
   ^
